In [1]:
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
import pandas as pd

In [2]:
# To do:
# Need to grab those 'what properties and views do i have access to functions
# Need to build a 'this can't change in a report' function (view, date, segment, etc.)
# And related, need to build a request list building
# Augment/build function to optionally return the max/min value thing
# Create clean/prep function that runs in output_to_df function that does things like standardize dimension names, etc.

def get_service_client(scopes, api_name, api_version, service_key_file=None, credentials=None):
    
    if credentials:
        service_client = build(api_name, api_version, credentials=credentials)
    
    else:
        credentials = ServiceAccountCredentials.from_json_keyfile_name(service_key_file, scopes=scopes)
        service_client = build(api_name, api_version, credentials=credentials)
    
    return service_client

def get_analytics_data(view_id, date_range, reporting_client, request_body):

    for request in request_body['reportRequests']:
        request.update({'viewId': view_id})
        request.update({'dateRanges': [date_range]})

    response_data = reporting_client.reports().batchGet(
        body=request_body
        ).execute()

    return response_data

def response_to_dataframe(response_data):
    
    data_frames = []
    
    # Yank out the meta data for each report
    for report in response_data.get('reports', []):
        headers = report.get('columnHeader', {})
        dimension_headers = headers.get('dimensions', [])
        metric_details = headers.get('metricHeader', {}).get('metricHeaderEntries', [])
        
        # Run through to grab metric headers
        metric_headers = []
        for metric in metric_details:
            metric_headers.append(metric.get('name'))
        
        # Pull dimension headers and metric headers together
        column_headers = tuple(dimension_headers + metric_headers)
        
        # For each report, yank out the actual data values
        rows_list = []
        for row in report.get('data').get('rows'):
            
            # Creates a tuple for each row of raw data
            row_data = []
            for dim_value in row.get('dimensions'):
                row_data.append(dim_value)

            for metric_value in row.get('metrics')[0].get('values'):
                row_data.append(metric_value)

            rows_list.append(tuple(row_data))

        # Build a data frame out of the data and append to list for multiple reports
        df = pd.DataFrame(rows_list, columns=column_headers)
        data_frames.append(df)
    
    return data_frames

In [3]:
scopes = ['https://www.googleapis.com/auth/analytics.readonly','https://www.googleapis.com/auth/analytics.edit']
service_credentials_file = 'client_secrets.json'
api_name = 'analyticsreporting'
api_version = 'v4'

reporting_client = get_service_client(scopes, api_name, api_version, service_credentials_file)

In [10]:
def get_analytics_data(view_id, date_range, reporting_client, request_body):

    for request in request_body['reportRequests']:
        request.update({'viewId': view_id})
        request.update({'dateRanges': [date_range]})

    response_data = reporting_client.reports().batchGet(
        body=(request_body
        ).execute()

    return response_data

# Tricky thing is this needs to be packaged so pull multiple requests into a single request body
# So how do I structure the parameter inputs
# We're going to pass this requests, lists that contain dictionaries with metrics and dimensions
def build_request_body(data_requests):
    
    for request in data_requests:
        
    
    return request_body

SyntaxError: invalid syntax (<ipython-input-10-e98eab0de21c>, line 11)

In [11]:
data_requests = [
    [{'metrics': ['sessions', 'pageviews'],
      'dimensions': ['country', 'browser']},
     {'metrics': ['sessions', 'pageviews', 'bounces'],
      'dimensions': ['fullReferrer', 'country', 'browser']}]]

data_requests

[[{'dimensions': ['country', 'browser'], 'metrics': ['sessions', 'pageviews']},
  {'dimensions': ['fullReferrer', 'country', 'browser'],
   'metrics': ['sessions', 'pageviews', 'bounces']}]]

In [4]:
view_id = '95100800'
date_range = {'startDate': '28daysAgo', 'endDate': 'today'}
request_body = {
        'reportRequests': [
            {
              'metrics': [{'expression': 'ga:sessions'}, {'expression': 'ga:pageviews'}],
              'dimensions': [{'name': 'ga:country'},{'name': 'ga:browser'}]
            },
            {
              'metrics': [{'expression': 'ga:sessions'}],
              'dimensions': [{'name': 'ga:browser'}, {'name': 'ga:country'}]
            },
            {
              'metrics': [{'expression': 'ga:sessions'}, {'expression': 'ga:sessions'}, {'expression': 'ga:bounces'}],
              'dimensions': [{'name': 'ga:browser'}, {'name': 'ga:country'}, {'name': 'ga:fullReferrer'}]
            }]
            }

response_data = get_analytics_data(view_id, date_range, reporting_client, request_body)

In [ ]:
analytics_df = response_to_dataframe(response_data)